#%% [markdown]

## Meteorological Visualisation App.

#%% [markdown]

# load modules

In [ ]:
import streamlit as st
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from windrose import WindroseAxes
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar

#%% [markdown] 

# Load wind data from NetCDF

In [ ]:
@st.cache_data
def load_temp_data():
    ds_temp = xr.open_dataset("data/air.2m.mon.ltm.1991-2020.nc")
    return ds_temp
@st.cache_data
def load_uwind_data():
    ds_u = xr.open_dataset("data/uwnd.mon.ltm.1991-2020.nc")
    return ds_u
@st.cache_data
def load_vwind_data():
    ds_v = xr.open_dataset("data/vwnd.mon.ltm.1991-2020.nc")
    return ds_v
@st.cache_data
def load_pr_data():
    ds_pr = xr.open_dataset("data/precip.mon.ltm.1991-2020.nc")
    return ds_pr
@st.cache_data
def load_rh_data():   
    ds_rh = xr.open_dataset("data/rhum.mon.ltm.1991-2020.nc")
    return ds_rh

2024-11-13 16:31:55.328 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.331 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.333 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.338 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.331 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.333 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.338 No runtime found, using MemoryCacheStorageManager
2024-11-13 16:31:55.340 No runtime found, using MemoryCacheStorageManager


#%% [markdown] 

 Function to select the spatial plot box

In [ ]:
def select_box(lat,lon):
    # Latitude and Longitude Box Selection
    if ((lat_min > lat_max) or lon_min > lon_max):
        st.text("ERROR: Minimum value greater than Maximum")    
    return lat_min, lat_max, lon_min, lon_max

#%% [markdown] 

 Function to plot windrose Plot

In [ ]:
def plot_wind_rose(speed_pwr, direction_pwr,lat_l,lon_l):
    fig_ws = plt.figure(figsize=(4, 4))
    ax = WindroseAxes.from_ax()
    ax.bar(direction_pwr, speed_pwr, normed=True, opening=0.8, edgecolor='white')
    ax.set_title('Latitude = '+str(lat_l)+' and Longitude = '+str(lon_l))
    ax.set_legend()
    ax.text(0.7,-0.2,'Data Source: '+ds_temp.attrs['source'],fontsize=4,transform=ax.transAxes)
    st.pyplot()  

#%% [markdown] 

 Function to plot wind vector plot

In [ ]:
def plot_spatial(temp_subset, lat_min, lat_max, lon_min, lon_max):
    # Select data within specified lat/lon box
    fig, ax3 = plt.subplots(nrows=2,ncols=6,subplot_kw={'projection': ccrs.PlateCarree()},
                           sharex=True,sharey=True,figsize=(12,4))
    for i_row in np.arange(2):
        for i_col in np.arange(6):
            ax3[i_row,i_col].set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
            ax3[i_row,i_col].add_feature(cfeature.COASTLINE)
            # ax.add_feature(cfeature.BORDERS)
            
            # Plot contour fill for wind speed
            lons, lats = np.meshgrid(temp_subset.lon, temp_subset.lat)
            s_plot = ax3[i_row,i_col].contourf(lons, lats, 
                                              np.squeeze(temp_subset.isel(time=int(i_row*6+i_col)))-273.15, 
                                              cmap='viridis', extend='both')
            ax3[i_row,i_col].set_title(calendar.month_name[int(i_row*6+i_col)+1][:3])
            
            if i_row==1:
                ax3[i_row,i_col].set_xlabel('Longitude')
                ax3[i_row,i_col].set_xticks(np.linspace(lon_min,lon_max,num=2,endpoint=True))
                ax3[i_row,i_col].set_xticklabels([lon_min,lon_max],fontsize=6)
                if i_col==0:
                    ax3[i_row,i_col].set_ylabel('Latitude')

            else: #0th Row
                if i_col==0:
                    ax3[i_row,i_col].set_ylabel('Latitude')
                    ax3[i_row,i_col].set_yticks(np.linspace(lat_min,lat_max,num=2,endpoint=True))
                else:                    
                    ax3[i_row,i_col].set_yticks(np.linspace(lat_min,lat_max,num=2,endpoint=True))
    
    fig.colorbar(s_plot, ax=ax3, label="2m Temperature (degC)", shrink=0.75)
    ax3.text(0.7,-0.2,'Data Source: '+ds_temp.attrs['source'],fontsize=4,transform=ax3.transAxes)
    st.pyplot(fig)

#%% [markdown]

Function to plot spatial variation in variables

In [ ]:
def plot_spatial2(var_subset,lat_min, lat_max, lon_min, lon_max,time_s):
    plt.rcParams['ytick.right'] = plt.rcParams['ytick.labelright'] = False
    fig, ax3 = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})#,

    ax3.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
    ax3.add_feature(cfeature.COASTLINE)
    
    # Plot contour fill for wind speed
    lons, lats = np.meshgrid(var_subset.lon, var_subset.lat)
    
    if var_subset.var_desc=='Air temperature':
        plot_data = np.squeeze(var_subset.isel(time=time_s))-273.15
        s_plot = ax3.contourf(lons,lats,plot_data,cmap='viridis', extend='both')
        fig.colorbar(s_plot, ax=ax3, label="2m Temperature (degC)", shrink=0.75)
        
    elif var_subset.var_desc=='Precipitation':
        plot_data = np.squeeze(var_subset.isel(time=time_s))
        s_plot = ax3.contourf(lons,lats,plot_data,cmap='viridis', extend='both')
        fig.colorbar(s_plot, ax=ax3, label="Mean Precipitation (mm/day)", shrink=0.75)
    else:
        plot_data = np.squeeze(var_subset.isel(time=time_s))
        s_plot = ax3.contourf(lons,lats,plot_data,cmap='viridis', extend='both')
        fig.colorbar(s_plot, ax=ax3, label="Relative humidity (%)", shrink=0.75)
        
    ax3.set_title(calendar.month_name[time_s][:3])
    ax3.set_xlabel('Longitude')
    ax3.set_ylabel('Latitude')

    ax3.set_xticks(np.linspace(lon_min,lon_max,num=5,endpoint=True))
    ax3.set_yticks(np.linspace(lat_min,lat_max,num=5,endpoint=True))
    ax3.text(0.7,-0.2,'Data Source: '+ds_temp.attrs['source'],fontsize=4,transform=ax3.transAxes)
    st.pyplot(fig)

#%% [markdown]

 Function to plot time series of variables

In [ ]:
def plot_time_series2(var): 
    # plt.rcParams['ytick.right'] = plt.rcParams['ytick.labelright'] = True
    fig, ax1 = plt.subplots(figsize=(12,6))
    color = 'tab:blue'
    if var.var_desc == 'Air temperature':        
        ax1.plot(np.arange(1,13), var-273.15,color=color)
        ax1.set_ylabel("2m Temperature (degC)",color=color, fontsize=14)
        ax1.set_yticks(np.linspace(np.floor(min(var.values - 273.15)),np.floor(max(var.values - 273.15)+1),num=5))
    elif var.var_desc == 'Precipitation':
        ax1.plot(np.arange(1,13), var,color=color)
        ax1.set_ylabel("Mean Precipitation (mm/day)",color=color, fontsize=14)
        ax1.set_yticks(np.linspace(np.floor(min(var.values)),np.floor(max(var.values)+1),num=5))
    else:
        ax1.plot(np.arange(1,13), var,color=color)
        ax1.set_xlabel("Month")
        ax1.set_ylabel("Relative humidity (%)",color=color, fontsize=14)
        ax1.set_yticks(np.linspace(np.floor(min(var.values)),np.floor(max(var.values)+1),num=5))
    
    ax1.set_xlabel("Month")
    ax1.set_xticks(np.arange(1,13))
    ax1.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug',
                        'Sep','Oct','Nov', 'Dec'])
    ax1.text(0.7,-0.2,'Data Source: '+ds_temp.attrs['source'],fontsize=4,transform=ax1.transAxes)
    st.pyplot(fig)

#%% [markdown]

 Function to covert 0 360 to -180 to 180

In [ ]:
def convert_180_180(ds_var):
    ds_var.coords['lon'] = (ds_var.coords['lon'] + 179.0625) % 358.125 - 179.0625
    ds_var2 = ds_var.sortby(ds_var.lon)
    return (ds_var2)

#%% [markdown]

 function to get the user input spatial box for plotting

In [ ]:
def user_input_box(lat,lon):
    lat_min = st.sidebar.number_input("Enter Lat. min.", min_value=float(str(lat.values.min())), 
                                    max_value=float(str(lat.values.max())), 
                                    value=-85.00,step=0.01, format='%2.2f')
    lat_max = st.sidebar.number_input("Enter Lat. max.", float(str(lat.values.min())), 
                                    max_value=float(str(lat.values.max())), 
                                    value=85.00,step=0.01, format='%2.2f',
                                    placeholder="Must be greater than Lat min.")
    lon_min = st.sidebar.number_input("Enter Lon. min.", min_value=float(str(lon.values.min())), 
                                    max_value=float(str(lon.values.max())), 
                                    value=-178.00,step=0.01, format='%3.2f')
    lon_max = st.sidebar.number_input("Enter Lon. max.", min_value=float(str(lon.values.min())), 
                                    max_value=float(str(lon.values.max())), 
                                    value=178.00,step=0.01, format='%3.2f',
                                    placeholder="Must be greater than Lon min.")
    return lat_min, lat_max, lon_min, lon_max

#%% [markdown]

 function to find lat, lon location for plotting

In [ ]:
def user_input_loc(lat,lon):
    lat_loc = st.sidebar.number_input("Enter Latitude", min_value=float(str(lat.values.min())), 
                                          max_value=float(str(lat.values.max())), 
                                          value=0.00,step=0.01, format='%2.2f')
    lon_loc = st.sidebar.number_input("Enter Longitude", min_value=float(str(lon.values.min())), 
                                          max_value=float(str(lon.values.max())), 
                                          value=0.00,step=0.01, format='%3.2f')
    return lat_loc, lon_loc

#%% [markdown] 

# Streamlit App

In [ ]:
st.title("Met. Data Visualization App")
ds_temp = convert_180_180(load_temp_data())
ds_u = convert_180_180(load_uwind_data())
ds_v = convert_180_180(load_vwind_data())
ds_pr = convert_180_180(load_pr_data())
ds_rh = convert_180_180(load_rh_data())
lon = ds_temp['lon']
lat = ds_temp['lat']

2024-11-13 16:37:35.404 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.409 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.405 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:35.406 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

#%% [markdown]

 User Inputs

In [ ]:
var_type = st.sidebar.selectbox("Choose the variable", ("Temp_2m", "Wind", "Precipitation","Relative Humidity"))

if var_type == 'Wind':
    plot_type = st.sidebar.selectbox("Choose Plot Type", ("Wind Rose", "Spatial Wind Vectors", "Time Series"))
    if plot_type == "Wind Rose":
        st.header("Wind Rose")
        lat_loc, lon_loc = user_input_loc(lat,lon)
        level_sel = st.sidebar.selectbox("Select Level (hPa)", ds_u.level.values)

        speed_loc, direction_loc = calculate_wind(ds_u['uwnd'].sel(lat=lat_loc,lon=lon_loc,method='nearest'),
                                          ds_v['vwnd'].sel(lat=lat_loc,lon=lon_loc,method='nearest'))

        plot_wind_rose(speed_loc.sel(level=level_sel).values, 
                       direction_loc.sel(level=level_sel).values,
                       speed_loc.lat.values,speed_loc.lon.values)

    elif plot_type == "Spatial Wind Vectors":
        st.header("Spatial Wind Vectors")        
        [lat_min, lat_max, lon_min, lon_max] = user_input_box(lat,lon)
        
        level_sel = st.sidebar.selectbox("Select Level (hPa)", ds_u.level.values)
        time_sel = st.sidebar.selectbox("Select Month", np.arange(1,13))
        
        ds_u_subset = ds_u['uwnd'].sel(lat=slice(lat_max, lat_min), 
                                       lon=slice(lon_min, lon_max),
                                       level=level_sel)
        ds_v_subset = ds_v['vwnd'].sel(lat=slice(lat_max, lat_min), 
                                       lon=slice(lon_min, lon_max),
                                       level=level_sel)
        plot_wind_vectors(ds_u_subset[time_sel-1,:,:], ds_u_subset[time_sel-1,:,:], 
                          lat_min, lat_max, lon_min, lon_max,time_sel)

    elif plot_type == "Time Series":
        st.header("Wind Speed Time Series")
        lat_loc, lon_loc = user_input_loc(lat,lon)
        level_sel = st.sidebar.selectbox("Select Level (hPa)", ds_u.level.values)
        speed_loc, direction_loc = calculate_wind(ds_u['uwnd'].sel(lat=lat_loc,lon=lon_loc,method='nearest'),
                                  ds_v['vwnd'].sel(lat=lat_loc,lon=lon_loc,method='nearest'))
        plot_time_series(speed_loc.sel(level=level_sel),direction_loc.sel(level=level_sel))
########################### 2m Temperature
elif var_type == 'Temp_2m':
    plot_type = st.sidebar.selectbox("Choose Plot Type", ("Spatial plot", "Time Series"))
    if plot_type == 'Spatial plot':
        st.header("Spatial plot")        
        [lat_min, lat_max, lon_min, lon_max] = user_input_box(lat,lon)
        ds_temp_subset = ds_temp['air'].sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
        time_sel = st.sidebar.selectbox("Select Month", np.arange(1,13))

        plot_spatial2(ds_temp_subset.sel(level=2), lat_min, lat_max, lon_min, lon_max,time_sel)

    else:
        st.header("Time series plot")
        lat_loc, lon_loc = user_input_loc(lat,lon)
        temp_loc = ds_temp['air'].sel(lat=lat_loc,lon=lon_loc,method='nearest')
        plot_time_series2(temp_loc.isel(level=0))
##################################### Precipitation        
elif var_type == 'Precipitation':
    plot_type = st.sidebar.selectbox("Choose Plot Type", ("Spatial plot", "Time Series"))
    if plot_type == 'Spatial plot':
        st.header("Spatial plot")
        [lat_min, lat_max, lon_min, lon_max] = user_input_box(lat,lon)        
        ds_pr_subset = ds_pr['precip'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
        time_sel = st.sidebar.selectbox("Select Month", np.arange(1,13))
        plot_spatial2(ds_pr_subset, lat_min, lat_max, lon_min, lon_max,time_sel)
    else:
        st.header("Time series plot")
        lat_loc, lon_loc = user_input_loc(lat,lon)
        pr_loc = ds_pr['precip'].sel(lat=lat_loc,lon=lon_loc,method='nearest')
        plot_time_series2(pr_loc)
######################################## Relative Humidity
else:
    plot_type = st.sidebar.selectbox("Choose Plot Type", ("Spatial plot", "Time Series"))
    if plot_type == 'Spatial plot':
        st.header("Spatial plot")        
        [lat_min, lat_max, lon_min, lon_max] = user_input_box(lat,lon)
        ds_rh_subset = ds_rh['rhum'].sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max))
        time_sel = st.sidebar.selectbox("Select Month", np.arange(1,13))
        level_sel = st.sidebar.selectbox("Select Level (hPa)", ds_rh.level.values)

        plot_spatial2(ds_rh_subset.sel(level=level_sel), lat_min, lat_max, lon_min, lon_max,time_sel)
    else:
        st.header("Time series plot")
        lat_loc, lon_loc = user_input_loc(lat,lon)
        rh_loc = ds_rh['rhum'].sel(lat=lat_loc,lon=lon_loc,method='nearest')
        level_sel = st.sidebar.selectbox("Select Level (hPa)", ds_rh.level.values)
        plot_time_series2(rh_loc.sel(level=level_sel))  

2024-11-13 16:37:46.486 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-13 16:37:46.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar